## Modèle 1 : SVM + optimisation des hyperparamètres avec `GridSearch`

In [1]:
from sklearn.datasets import fetch_20newsgroups
groups = fetch_20newsgroups()

In [2]:
data_train = fetch_20newsgroups(subset='train', random_state=21)
train_label = data_train.target
data_test = fetch_20newsgroups(subset='test', random_state=21)
test_label = data_test.target
len(data_train.data), len(data_test.data), len(test_label)

(11314, 7532, 7532)

In [3]:
import numpy as np
np.unique(test_label)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])

On a 20 classes différentes

## **Nettoyage du corpus et réduction du bruit dans ce dernier**

On filtre les éléments dans le corpus avec 2 conditions :

#### **1. Suppression des entités nommées**

Les prénoms peuvent introduire du bruit et biaiser l'apprentissage du modèle. Pour retirer les prénoms, on utilise le corpus NLTK qui contient de très nombreux noms en anglais. Ensuite on retire les noms dans la boucle, en utilisant `words not in all_names`.

#### **2. Suppression des caractères spéciaux**

On élimine la ponctuation (ex: "!", "?", ".") et les nombres avec `words.isalpha()`.


#### **Normalisation et lemmatisation**

Si un mot satisfait ces deux conditions, on applique `words.lower()` qui transforme le mot en minuscules et on réduit le mot à sa forme de base (lemme) avec `WNL.lemmatize(...)` . Cela réduit la dimensionalité du vocabulaire








In [4]:
import nltk
from collections import defaultdict
from nltk.stem import WordNetLemmatizer
from nltk.corpus import names

nltk.download('names')
nltk.download('wordnet')
nltk.download('omw-1.4')  # OMW est l'acronyme de "Open Multilingual Wordnet"

all_names = set(names.words())
WNL = WordNetLemmatizer()

def clean(data):
    cleaned = defaultdict(list)
    count = 0
    for group in data:
        for words in group.split():
            if words.isalpha() and words not in all_names:
                cleaned[count].append(WNL.lemmatize(words.lower()))
        cleaned[count] = ' '.join(cleaned[count])
        count += 1
    return list(cleaned.values())

[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Unzipping corpora/names.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [5]:
x_train = clean(data_train.data)
x_train[0]

'bouncing lymenet lehigh university the following address are on the lymenet mailing but are rejecting since the list server originally accepted these address i assume these address have since been improperly functioning mail gateway might also be if you are listed here and would still like to remain on the please write to i will remove these address from the list before the next newsletter go a a general please remember to from all your mailing list before your account is this will save the listserv maintainer from many box lehigh university'

In [6]:
len(x_train)

11314

In [7]:
x_test = clean(data_test.data)
len(x_test)

7532

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(stop_words='english', max_features=1000)
X_train = tf.fit_transform(x_train)
X_test = tf.transform(x_test)
X_train.shape, X_test.shape

((11314, 1000), (7532, 1000))

Cet output signifie qu'on a:
* 11314 documents dans $X_{train}$
* 7532 documents dans $X_{test}$

Chaque document est représenté par un vecteur de 1000 features TF-IDF

# **SVM linéaire**


On distingue deux grands types de SVM linéaires : le **Hard margin** SVM et le **Soft margin** SVM.

### SVM **Hard Margin**

Dans le premier, les instances doivent être parfaitement séparables par un hyperplan linéaire.

Pour les points $x_i$ dans la classe $y_i = 1$ on impose :
$$\omega^\top x_i + b \ge 1$$

Pour les points $x_i$ dans la classe $y_i = -1$ on impose :
$$\omega^\top x_i + b \le -1$$

On constate qu'un élément $x_i$ est bien classifié si $y_i$ et $(\omega^\top x_i + b)$ ont le même signe. Ainsi, on peut reformuler ces deux contraintes de manière unifiée :
$$\boxed{\forall i,\quad y_i \cdot (\omega^\top x_i + b) \ge 1}$$


Mais cela suppose que les données soient parfaitement linéairement séparables.

Si ce n'est pas le cas, alors, quel que soit l’hyperplan choisi, il existera au moins un point $x_i$ pour lequel $y_i \cdot (\omega^\top x_i + b) \le 0$ c’est-à-dire au moins un point mal classé (ce qui arrive si $y_i$ et $(\omega^\top x_i + b)$ ont des signes différents), ou situé sur la frontière de décision (ce qui arrive si $y_i \cdot (\omega^\top x_i + b) = 0$).

Formellement, c'est linéairement séparable si

$$\exists (\omega,b)\ \text{tel que}\ \forall i,\ y_i(\omega^\top x_i + b) > 0$$


et non linéairement séparable si :

$$\forall (\omega,b),\ \exists i\ \text{tel que}\ y_i(\omega^\top x_i + b) \le 0$$

### SVM **Soft Margin**

C'est ce pourquoi on utilise un **SVM linéaire soft margin** qui fonctionne même si les données ne sont pas parfaitement linéairement séparables en tolérant des écarts et donc des erreurs de classifications éventuelles.

De manière générale, dans un SVM, il faut maximiser l'espacement entre les deux marges (qui correspondent aux deux hyperplans $\{ x \in \mathbb{R}^d \;|\; \omega^\top x + b = 1 \}$ et $\{ x \in \mathbb{R}^d \;|\; \omega^\top x + b = -1 \}$). Mais, avec certaines données non linéairement séparables, il faut trouver un compromis entre la largeur de la marge et les erreurs de classification.

Dans un SVM soft margin, on associe à chaque point $x_i$ une quantité $\xi_i$ qui indique “de combien il manque” pour satisfaire la condition idéale $y_i(\omega^\top x_i + b) \ge 1$. Pour chaque point $x_i$, la quantité $\xi_i$ est donnée par :

$$\xi_i = \max(0, 1 - y_i (\omega^\top x_i + b))$$

Il y a trois cas possibles (bien classé, dans la marge, mal classé) :

* Si le point $x_i$ est bien classé :

$$
y_i (\omega^\top x_i + b) \ge 1
\quad\Rightarrow\quad
\big[1 - y_i (\omega^\top x_i + b)\big] \le 0
\quad\Rightarrow\quad
\xi_i = 0
$$

* Si le point $x_i$ est dans la marge (entre les deux hyperplans) mais correctement classé :

$$
0 < y_i (\omega^\top x_i + b) < 1
\quad\Rightarrow\quad
0 < \xi_i = \big[1 - y_i (\omega^\top x_i + b)\big] < 1
$$

* Si le point $x_i$ est mal classé :

$$
y_i (\omega^\top x_i + b) < 0
\quad\Rightarrow\quad
\xi_i = \big[1 - y_i (\omega^\top x_i + b)\big] > 1
$$

Maintenant, on peut pénaliser le modèle à la fois en fonction du nombre de points du mauvais côté (en comptant à la fois les points dans les marges bien classés et ceux qui sont mal classés) et de l'ampleur des violations (plus la position d'un point s'écarte de sa position attendue, plus la pénalité sur la loss sera forte).

Pour ce faire, on dispose de l'hyperparamètre $C$ qui gère le compromis entre la largeur de la marge et les erreurs sur l’échantillon.

Si $C$ est grand, on cherche à réduire au maximum ces erreurs quitte à avoir une marge plus petite ; si $C$ est petit, on tolère davantage les violations pour favoriser une marge plus large. Dans ce qui suit, on teste 4 valeurs de $C$ avec `GridSearchCV`.


### Sources :

https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

https://en.wikipedia.org/wiki/Support_vector_machine#Linear_SVM


In [9]:
from sklearn.svm import SVC
svc_lib = SVC(kernel = 'linear')
parameters = {'C' : (0.5,1.0,10,100)}

In [10]:
from sklearn.model_selection import GridSearchCV
import timeit

grid_search1 =GridSearchCV(svc_lib, parameters, n_jobs = -1, cv = 3)
start_time = timeit.default_timer()
grid_search1.fit(X_train, train_label)
final = timeit.default_timer()-start_time
print("Temps d'execution : ",final)

Temps d'execution :  144.68969013400002


In [11]:
from sklearn.svm import LinearSVC

linear_svc = LinearSVC()
linear_svc.fit(X_train, train_label)

print("coef_.shape :", linear_svc.coef_.shape)
print("intercept_.shape :", linear_svc.intercept_.shape)

coef_.shape : (20, 1000)
intercept_.shape : (20,)


In [12]:
print(grid_search1.best_params_)
print(grid_search1.best_score_)

{'C': 1.0}
0.7218490556805537


In [13]:
grid_search_best1 = grid_search1.best_estimator_
accur1 = grid_search_best1.score(X_test, test_label)
print(accur1)

0.6269251194901753


In [14]:
print("coef_.shape :", linear_svc.coef_.shape)
print("intercept_.shape :", linear_svc.intercept_.shape)

coef_.shape : (20, 1000)
intercept_.shape : (20,)


In [15]:
import numpy as np
print("Nb classes dans y_train :", len(np.unique(test_label)))


Nb classes dans y_train : 20


In [16]:
import numpy as np

print("decision_function_shape:", svc_lib.decision_function_shape)

decision_function_shape: ovr


OvR est l'abréviation de **One-vs-Rest** : on entraîne 1 classifieur par classe (classe $k$ vs toutes les autres) → n_classes hyperplans.

Il semble que c'est plus rapide que la méthode OvO **One versus one** qui cherche $\frac{n(n-1)}{2}$  hyperplans différents, donc ici puisque $n= 20$ cela chercherait $\frac{20×19}{2}=190$  hyperplans.

In [25]:
from sklearn.svm import SVC
import numpy as np

svc_lib = SVC(kernel='linear')

svc_lib.fit(X_train, train_label)

print("decision_function_shape:", svc_lib.decision_function_shape)
print("classes:", svc_lib.classes_)
print("n_support per class:", svc_lib.n_support_)
print("support_vectors_.shape:", svc_lib.support_vectors_.shape)
print("dual_coef_.shape:", svc_lib.dual_coef_.shape)

if hasattr(svc_lib, "coef_"):
    print("coef_.shape:", svc_lib.coef_.shape)

print("intercept_.shape:", svc_lib.intercept_.shape)

print("decision_function(X_test).shape:", svc_lib.decision_function(X_test).shape)

n = len(svc_lib.classes_)
print("il y a théoriquement", n*(n-1)//2, "hyperplans")

decision_function_shape: ovr
classes: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
n_support per class: [421 487 467 497 507 470 446 427 420 458 355 310 509 442 407 471 371 306
 402 368]
support_vectors_.shape: (8541, 1000)
dual_coef_.shape: (19, 8541)
coef_.shape: (190, 1000)
intercept_.shape: (190,)
decision_function(X_test).shape: (7532, 20)
il y a théoriquement 190 hyperplans


# Linear SVC

In [18]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
import timeit

linear_svc = LinearSVC()
parameters = {'C': (0.5, 1, 10,100)}

grid_search2 = GridSearchCV(linear_svc, parameters, n_jobs = -1, cv = 3)
start_time = timeit.default_timer()
grid_search2.fit(X_train, train_label)
final = timeit.default_timer()-start_time
print("Execution Time : ",final)

Execution Time :  6.762827636999987


In [20]:
linear_svc = LinearSVC()
linear_svc.fit(X_train, train_label)
print(linear_svc.coef_.shape)
print(linear_svc.intercept_.shape

(20, 1000)
(20,)


Cela signifie qu'il y a **20 hyperplans différents**

In [21]:
print(grid_search2.best_params_)
print(grid_search2.best_score_)

{'C': 0.5}
0.7244122673837164


In [22]:
grid_search_best2 = grid_search2.best_estimator_
accur2 = grid_search_best2.score(X_test, test_label)
accur2

0.6314391927774827

# Model Tuning -> Linear SVC

In [23]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([('tf_id', TfidfVectorizer(stop_words = "english")), ('svm_im', LinearSVC())])
pipeline

parameter = {'tf_id__max_features' : (1000, 2000, 8000),
             'tf_id__max_df' : (0.25, 0.5),
             'tf_id__smooth_idf' : (True, False),
             'tf_id__sublinear_tf' : (True, False)
}

On teste toutes les combinaisons d’hyperparamètres définies dans parameter avec une [Cross Validation](https://scikit-learn.org/stable/modules/cross_validation.html) à 3 folds. Par défaut `refit=True`, on réentraine le meilleur modèle (celui avec le meilleur score moyen en CV) sur tout $X_train$

In [24]:
grid_search = GridSearchCV(pipeline, parameter,cv = 3)
grid_search.fit(x_train, train_label)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('tf_id',
                                        TfidfVectorizer(stop_words='english')),
                                       ('svm_im', LinearSVC())]),
             param_grid={'tf_id__max_df': (0.25, 0.5),
                         'tf_id__max_features': (100, 1000, 2000, 8000),
                         'tf_id__smooth_idf': (True, False),
                         'tf_id__sublinear_tf': (True, False)})

In [26]:
print(grid_search.best_params_)

{'tf_id__max_df': 0.25, 'tf_id__max_features': 8000, 'tf_id__smooth_idf': False, 'tf_id__sublinear_tf': True}


In [27]:
print(grid_search.best_score_)

0.8699845024806998


In [28]:
#@title Régression Logistique

from sklearn.linear_model import LogisticRegression

# Solver 'liblinear' est bon pour les petits datasets, 'saga' pour les très gros
log_reg = LogisticRegression(solver='liblinear', multi_class='ovr', max_iter=1000)

pipeline_lr = Pipeline([
    ('tf_id', TfidfVectorizer(stop_words="english", max_features=5000)),
    ('clf', log_reg)
])

pipeline_lr.fit(x_train, train_label)
print("Score LogReg:", pipeline_lr.score(x_test, test_label))

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


Score LogReg: 0.758364312267658
